# Using SEM to analyze the output of ns-3 simulations

This notebook provides an overview of features offered by SEM.
In particular, we will:

1. Create an ns-3 simulation campaign
2. Use it to run some simulations
3. Parse the results
4. Explore how input parameters change the output of our simulations

First of all, let's import the library, which can easily be installed through pip:

```bash
pip3 install sem
```

In [ ]:
import sem

import pprint

Now, let's create a campaign object. In order to do this, we need to tell SEM about three things:

1. The location of our ns-3 installation;
2. The name of the script to run in this campaign;
3. A directory in which to save results.

For this example, we will employ the `wifi-multi-tos` script, which prints the aggregated throughput of a WiFi network to the standard output, and which can be configured through various command-line parameters.

In [ ]:
script = 'wifi-multi-tos'
ns_3_dir = './ns-3'
results_dir = 'results'
campaign = sem.CampaignManager.new(ns_3_dir, script, results_dir)

Note that this step also compiled ns-3 in optimized mode. We can now print the newly obtained object to inspect it:

In [ ]:
print(campaign)

As a next step, let's define the space of the parameters we are interested in exploring. A list of all available parameters is already provided when printing the campaign info: we just need to create a dictionary where the keys are the parameter names and the values are lists of values for the corresponding parameter. Additionally, we also define a `runs` variable, in which we specify how many ranodmized runs we want to perform for each parameter combination.

In [ ]:
param_ranges = {
    'channelWidth': [20],  # Channel width of 20 MHz
    'distance': [10],  # Distance from the Access Point
    'mcs': [0, 3, 6],  # Modulation Coding Scheme to employ
    'nWifi': [1, 2],  # Number of wifi devices to connect to the AP
    'simulationTime': [2],  # Duration of the simulation in seconds
    'useRts': [False, True],  # Whether to use the Request To Send mechanism
    'useShortGuardInterval': [False, True]  # Whether to use the short guard interval
}
runs = 3

We can now run the simulations specified by the parameter space.

In [ ]:
campaign.run_missing_simulations(param_ranges, runs)

Now, try executing the cell above a second time: you will notice that the execution terminates immediately. SEM knows it already performed the simulations we asked for, and that they are saved in the local database - there's no need to repeat them, so the call just terminates. If, instead, you increase the value of the `runs` variable, SEM will need to run additional simulations when the simulation running function is called again.

Let's inspect one of the results we got from the simulation: we can do this by calling the `get_complete_results` function provided by the campaign's `db` object. When called without arguments, this function returns a list of all available results - we just print the first one.

In [ ]:
example_result = campaign.db.get_complete_results()[0]
pprint.pprint(example_result)

We can see that the result object is simply a nested dictionary structure.

- The `meta` key contains the time it took to run the simulation, and a unique id for the result;
- The `output` key contains a list of all files generated by the simulation, and their contents;
- The `params` key contains all the parameters that were used to obtain the result.

The value we are interested in is the throughput, and it is contained in the `stdout` entry of the `output` dictionary. Let's use the `example_result` to test a function that extracts this value from the result.

In [ ]:
print ("Raw output string: %s" % example_result['output']['stdout'])

def get_throughput(result):
    if "Aggregated throughput" in result['output']['stdout']:
            return float(result['output']['stdout'].split(" ")[-2])
    return 0

print("Parsed result: %s" % get_throughput(example_result))

The `get_throughput` function we just defined and tested operates on a result, tests whether the 'Aggregated throughput' line is in it, and then extracts the second-to-last element of the sentence, which corresponds to the desired throughput value.

Now that we have a function that translates simulation results into a metric of interest (throughput in our case), we can employ it to explore how the input parameters influence throughput. For this, SEM provides the `utils.interactive_plot` function, which requires as inputs the simulation campaign object, a range of parameters we want to explore, a result-parsing function, the parameter to put on the horizontal axis and the number of runs to average before plotting. If the function is run from within a Jupyter notebook, an interactive plot with interactive parameter-selection widgets is generated.

In [ ]:
sem.utils.interactive_plot(campaign, param_ranges, get_throughput, 'mcs', runs)